In [2]:
#% pylab inline
import os
import random

import pandas as pd
from scipy.misc import imread


In [4]:
root_dir = os.path.abspath('.')
data_dir = '/media/sayak/Local Disk/AV/data'

train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [5]:
#%pylab inline
from scipy.misc import imshow
i = random.choice(train.index)

img_name = train.ID[i]
img = imread(os.path.join(data_dir, 'Train', img_name))


imshow(img)
print("Age:" + train.Class[i])

Age:YOUNG


In [6]:
from scipy.misc import imresize
import numpy as np

temp = []
for img_name in train.ID:
    img_path = os.path.join(data_dir, 'Train', img_name)
    img = imread(img_path)
    img = imresize(img, (32, 32))
    img = img.astype('float32') # this will help us in later stage
    temp.append(img)

train_x = np.stack(temp)

In [7]:
temp = []
for img_name in test.ID:
    img_path = os.path.join(data_dir, 'Test', img_name)
    img = imread(img_path)
    img = imresize(img, (32, 32))
    temp.append(img.astype('float32'))

test_x = np.stack(temp)

In [9]:
train_x = train_x / 255.
test_x = test_x / 255.

In [10]:
import keras
from sklearn.preprocessing import LabelEncoder


lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.np_utils.to_categorical(train_y)


Using TensorFlow backend.


In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils


input_num_units = (32, 32, 3)
#hidden_num_units = 500
#output_num_units = 3

epochs = 30
batch_size = 64

model = Sequential()
model.add(Conv2D(50, (5, 5), input_shape=(input_num_units), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(40, (5, 5), input_shape=(input_num_units), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(30, (5, 5), input_shape=(input_num_units), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1,validation_split = 0.2)


Train on 15924 samples, validate on 3982 samples
Epoch 1/30
15924/15924 [==============================] - 75s - loss: 0.9646 - acc: 0.5419 - val_loss: 0.9501 - val_acc: 0.5439
Epoch 2/30
15924/15924 [==============================] - 71s - loss: 0.9536 - acc: 0.5425 - val_loss: 0.9426 - val_acc: 0.5439
Epoch 3/30
15924/15924 [==============================] - 75s - loss: 0.9386 - acc: 0.5426 - val_loss: 0.9395 - val_acc: 0.5439
Epoch 4/30
15924/15924 [==============================] - 74s - loss: 0.9174 - acc: 0.5446 - val_loss: 0.8964 - val_acc: 0.5525
Epoch 5/30
15924/15924 [==============================] - 71s - loss: 0.8946 - acc: 0.5661 - val_loss: 0.9028 - val_acc: 0.5585
Epoch 6/30
15924/15924 [==============================] - 69s - loss: 0.8814 - acc: 0.5801 - val_loss: 0.8679 - val_acc: 0.5816
Epoch 7/30
15924/15924 [==============================] - 67s - loss: 0.8686 - acc: 0.5878 - val_loss: 0.9010 - val_acc: 0.5635
Epoch 8/30
15924/15924 [==============================]

In [29]:
pred = model.predict_classes(test_x)
pred = lb.inverse_transform(pred)

test['Class'] = pred
test.to_csv('sub02.csv', index=False)

6624/6636 [============================>.] - ETA: 0s